In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import gzip
from scipy import stats
from subprocess import check_output
%matplotlib inline

In [ ]:
production_df = pd.read_excel('../data/tmdb_production_data.xlsx', index_col=0)
movie_df = pd.read_excel('../data/tmdb_movie_data.xlsx', index_col=0)
genre_df = pd.read_excel('../data/movie_genres_data.xlsx', index_col=0)

In [ ]:
genre_list = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']

In [ ]:
movie_production_df = production_df.merge(movie_df, how='inner', on='movie_title')
drop_budget = movie_production_df.index[movie_production_df["movie_budget"] == 0].tolist()
drop_revenue = movie_production_df.index[movie_production_df["movie_revenue"] == 0].tolist()
drop_both = drop_budget+drop_revenue
movie_production_df = movie_production_df.drop(movie_production_df.index[drop_both])
movie_production_df = movie_production_df.sort_values('movie_vote_count', ascending=False).drop_duplicates('movie_id').sort_index()
movie_production_df

In [ ]:
rev_pop = movie_production_df[['movie_revenue', 'movie_popularity']]
rev_pop['movie_popularity'] = round(rev_pop['movie_popularity'], 0)
rev_pop.reset_index(inplace=True)
rev_pop.drop(columns='index', inplace=True)
rev_pop

In [ ]:
z = np.abs(stats.zscore(rev_pop['movie_popularity']))
outliers = (np.where(z > 3))[0]
outliers

In [ ]:
for x in outliers:
    rev_pop.drop([x], axis=0, inplace=True)
rev_pop

In [ ]:
rev_pop['normalized_popularity'] = (rev_pop['movie_popularity'] - rev_pop['movie_popularity'].min()) / (rev_pop['movie_popularity'].max() - rev_pop['movie_popularity'].min()) 
rev_pop['normalized_popularity'] = round(rev_pop['normalized_popularity'] * 100, 0)
rev_pop

In [ ]:
test = rev_pop[['normalized_popularity', 'movie_revenue']]
test['normalized_popularity'] = rounded = round(test['normalized_popularity']/10)*10
test = test.groupby('normalized_popularity').mean()
test['movie_revenue'] = round(test['movie_revenue']/1000000, 0)
test

In [ ]:
test['movie_revenue'].plot(kind = 'bar')